# GIC Analytics
## Cleaning data (Fixing data types for datetime)

In [28]:
import pandas as pd
import numpy as np
import json
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

market_values = pd.read_csv('market-values-cleaned.csv')
instruments = pd.read_csv('instruments-cleaned.csv')
transactions = pd.read_csv('transactions-cleaned.csv')

pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Requirements


### Basic: View cumulative transaction amount, cumulative quantity based on instrument ID

In [22]:
def view_instrument_data(instrID, transactions):
    now = pd.Timestamp.now()
    transactions['transactionDate']= pd.to_datetime(transactions['transactionDate'])
    transactions = transactions.loc[transactions['isCancelled'] == False]
    transactions = transactions.loc[transactions['instrumentId'] == instrID]
    transactions = transactions.loc[transactions['transactionDate'] < now]
    transactions_filtered = transactions[['quantity', 'transactionAmount', 'instrumentId']]
    
    results = transactions_filtered.groupby('instrumentId').sum()
    result = results.to_json(orient="records")
    parsed = json.loads(result)
    return json.dumps(parsed, indent=4)
    
view_instrument_data(152, transactions.copy(deep=True))


'[\n    {\n        "quantity": 83.0,\n        "transactionAmount": -141105813.75\n    }\n]'

### 2. View total market value of a single investment across a date range

In [24]:
def view_single_marketvalue(start, end, instrID, market_values, instruments):
    market_values['marketValueDate']= pd.to_datetime(market_values['marketValueDate'])
    start_date = pd.Timestamp(start)
    end_date = pd.Timestamp(end)
    merged_df = pd.merge(left=market_values, right=instruments, left_on='instrumentId', right_on='instrumentId')
    merged_df = merged_df[['instrumentId', 'instrumentName', 'marketValue', 'marketValueDate']]
    merged_df = merged_df.loc[merged_df['instrumentId'] == instrID]
    merged_df = merged_df.loc[(merged_df['marketValueDate'] > start_date) & (merged_df['marketValueDate'] < end_date)]

    result = merged_df.to_json(orient="records")
    parsed = json.loads(result)
    return json.dumps(parsed, indent=4)
    
    
    
    
view_single_marketvalue("20200917", "20221217", 152, market_values.copy(deep=True), instruments.copy(deep=True))

'[\n    {\n        "instrumentId": 152,\n        "instrumentName": "Sanford Tremblay and Wunsch",\n        "marketValue": 120000000.0,\n        "marketValueDate": 1601424000000\n    },\n    {\n        "instrumentId": 152,\n        "instrumentName": "Sanford Tremblay and Wunsch",\n        "marketValue": 120000000.0,\n        "marketValueDate": 1604102400000\n    },\n    {\n        "instrumentId": 152,\n        "instrumentName": "Sanford Tremblay and Wunsch",\n        "marketValue": 120000000.0,\n        "marketValueDate": 1606694400000\n    },\n    {\n        "instrumentId": 152,\n        "instrumentName": "Sanford Tremblay and Wunsch",\n        "marketValue": 147615415.42,\n        "marketValueDate": 1609372800000\n    },\n    {\n        "instrumentId": 152,\n        "instrumentName": "Sanford Tremblay and Wunsch",\n        "marketValue": 147615415.42,\n        "marketValueDate": 1612051200000\n    },\n    {\n        "instrumentId": 152,\n        "instrumentName": "Sanford Tremblay and

### 3. View the breakdown (by investments) of the net profit/loss of all investments as of a certain date

In [29]:
def nearest(items, pivot):
    return pd.to_datetime(min([i for i in items], key=lambda x: abs(x - pivot)))

def view_multiple_profitloss(date, market_values, transactions):
    end_date = pd.Timestamp(date)
    market_values['marketValueDate']= pd.to_datetime(market_values['marketValueDate'])
    market_values_less = market_values.loc[market_values['marketValueDate'] <= end_date]
    instrument_ids_list = list(set(market_values_less['instrumentId'].tolist()))
    market_values_all_instruments = pd.DataFrame(columns = ['instrumentId', 'marketValueDate', 'marketValue', 'createdAt'])
    for id in instrument_ids_list:
        instrument_marketValues = market_values_less.loc[market_values['instrumentId']==id]
        closest_value_idx = instrument_marketValues.loc[instrument_marketValues['marketValueDate'] == nearest(instrument_marketValues['marketValueDate'].to_list(),end_date)].index[0]
        market_values_all_instruments = market_values_all_instruments.append(market_values.iloc[closest_value_idx])

    transactions['transactionDate']= pd.to_datetime(transactions['transactionDate'])
    transactions = transactions.loc[(transactions['transactionDate'] < end_date)]
    transactions = transactions[['instrumentId', 'transactionAmount', 'quantity', 'transactionDate']]
    
    cumulative_transactions = pd.DataFrame(transactions.groupby('instrumentId').sum())
    #print(cumulative_transactions)
    market_values_all_instruments['netprofitloss'] = 0
    for id in instrument_ids_list:
        marketValue = (market_values_all_instruments.loc[market_values_all_instruments['instrumentId']==id])['marketValue']
        quantity = (cumulative_transactions.iloc[id-1])['quantity']
        transaction_amount = (cumulative_transactions.iloc[id-1])['transactionAmount']
        market_values_all_instruments.iloc[[id-1],[5]]= marketValue * (quantity/100.0) + transaction_amount
    #print(market_values_all_instruments)
    results = market_values_all_instruments[['netprofitloss', 'instrumentId']]
    result = results.to_json(orient="records")
    parsed = json.loads(result)
    return json.dumps(parsed, indent=4)
    
view_multiple_profitloss("20220430", market_values.copy(deep=True), transactions.copy(deep=True))

'[\n    {\n        "netprofitloss": 0,\n        "instrumentId": 1\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 2\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 3\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 4\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 5\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 6\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 7\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 8\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 9\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 10\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 11\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 12\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 13\n    },\n    {\n        "netprofitloss": 0,\n        "instrumentId": 14\n    }

### 4. View net profit/loss of a single investment across a date range

In [30]:
def view_single_profitloss(start, end, instrID, market_values, transactions):
    start_date = pd.Timestamp(start)
    end_date = pd.Timestamp(end)
    market_values['marketValueDate']= pd.to_datetime(market_values['marketValueDate'])
    market_values = market_values.loc[market_values['instrumentId'] == instrID]
    market_values = market_values.loc[(market_values['marketValueDate'] > start_date) & (market_values['marketValueDate'] < end_date)]
    market_values = market_values.reset_index()
    date_array = market_values['marketValueDate'].tolist()
    market_values["cumulative_quantity"] = ""
    market_values["asset_value"] = ""
    market_values["cumulative_transaction_amount"] = ""
    market_values["net_profitloss"] = ""
    
    
    transactions = transactions.loc[transactions['instrumentId'] == instrID]
    transactions['transactionDate']= pd.to_datetime(transactions['transactionDate'])
    transactions = transactions.loc[(transactions['transactionDate'] < end_date)]
    transactions = transactions[['instrumentId', 'transactionAmount', 'quantity', 'transactionDate']]
    count = 0
    for each in date_array:
        transactions_filtered = transactions.loc[(transactions['transactionDate'] <= each)]
        transactions_filtered = transactions_filtered.groupby(['instrumentId']).sum()
        market_values.at[count,"cumulative_quantity"] = transactions_filtered['quantity'].iloc[0]
        if np.isnan(market_values.at[count, "cumulative_quantity"]):
            market_values.at[count, "cumulative_quantity"] = 0
        market_values.at[count,"cumulative_transaction_amount"] = transactions_filtered['transactionAmount'].iloc[0]
        market_values.at[count,"asset_value"] = (int(float(market_values["cumulative_quantity"].iloc[count]))/100) * market_values["marketValue"].iloc[count]
        market_values.at[count,"net_profitloss"] = market_values["cumulative_transaction_amount"].iloc[count] + market_values["asset_value"].iloc[count]
        count +=1
        
    results = market_values[['marketValueDate', 'net_profitloss']]
    result = results.to_json(orient="records")
    parsed = json.loads(result)
    return json.dumps(parsed, indent=4)
    
    
    
view_single_profitloss("20201117", "20221217", 152, market_values, transactions)
    
    
    

/var/folders/1p/n8fy0yrn49x99sx5h2hkvrtr0000gn/T/ipykernel_35476/2701344785.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions['transactionDate']= pd.to_datetime(transactions['transactionDate'])


'[\n    {\n        "marketValueDate": 1606694400000,\n        "net_profitloss": -3408000.0\n    },\n    {\n        "marketValueDate": 1609372800000,\n        "net_profitloss": -22458.4580000006\n    },\n    {\n        "marketValueDate": 1612051200000,\n        "net_profitloss": -22458.4580000006\n    },\n    {\n        "marketValueDate": 1614470400000,\n        "net_profitloss": -22458.4580000006\n    },\n    {\n        "marketValueDate": 1617148800000,\n        "net_profitloss": 2810830.976\n    },\n    {\n        "marketValueDate": 1619740800000,\n        "net_profitloss": 2810830.976\n    },\n    {\n        "marketValueDate": 1622419200000,\n        "net_profitloss": 2810830.976\n    },\n    {\n        "marketValueDate": 1625011200000,\n        "net_profitloss": 910589.227199994\n    },\n    {\n        "marketValueDate": 1627689600000,\n        "net_profitloss": 910589.227199994\n    },\n    {\n        "marketValueDate": 1630368000000,\n        "net_profitloss": 910589.227199994\n  